In [ ]:
import os
import ee
import datetime
import time
import sklearn
import io
import requests
import urllib.request

import geopandas as gp
import numpy as np
import pandas as pd
import rsfuncs as rs
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from scipy.signal import savgol_filter
from dateutil.relativedelta import relativedelta
from sklearn import preprocessing
from shapely.ops import unary_union
from pandas.tseries.offsets import MonthEnd

ee.Initialize()


In [ ]:
# Select area

shp = gp.read_file("../shape/argus_grace.shp")
sa_km3 = round(shp.area[0] * 10000, 2)
area = rs.gdf_to_ee_poly(shp)


In [ ]:
# Set GRACE study period 

years = range(2003, 2018)
months = range(1,13)
start = datetime.datetime(years[0], 1, 1)
end = datetime.datetime(years[-1]+1, 1, 1)
dt_idx = pd.date_range(start,end, freq='M')

In [ ]:
# Load data
data = rs.load_data()

In [ ]:
# Get grace producgts
cri = rs.grace_wrapper(data['cri'])
mas = rs.grace_wrapper(data['mas'])
gfz = rs.grace_wrapper(data['gfz'])
csr = rs.grace_wrapper(data['csr'])
jpl = rs.grace_wrapper(data['jpl'])

In [ ]:
# Now soil moisture, SWE, and reservoir storage. 
# Add an extra year beforehand, since we are differencing from the starting point, (e.g. dS/dt) 

In [ ]:
# Soil moisture

tc_sm = rs.calc_monthly_sum(data['tc_sm'], range(2002, 2018), months, area)
gldas_sm = rs.calc_monthly_sum(data['gldas_sm'], range(2002, 2018), months, area)

# gldsm1 = rs.calc_monthly_sum(data['gsm1'], years, months, area)
# gldsm2 = rs.calc_monthly_sum(data['gsm2'], years, months, area)
# gldsm3 = rs.calc_monthly_sum(data['gsm3'], years, months, area)
# gldsm4 = rs.calc_monthly_sum(data['gsm4'], years, months, area)

In [ ]:
# SWE

fldas_swe = rs.calc_monthly_sum(data['fldas_swe'], range(2002, 2018), months, area)
gldas_swe = rs.calc_monthly_sum(data['gldas_swe'], range(2002, 2018), months, area)

# dmet_swe = rs.calc_monthly_sum(data['dmet_swe'], years, months, area)

In [ ]:
# Reservoir Storage

reservoirs = gp.read_file("../shape/cdec_reservoirs.shp")
within_GRACE = gp.sjoin(reservoirs, shp, how='inner', op='within')

res_data = {}

for i in within_GRACE.ID:
    print("processing " + i )
    url = "https://cdec.water.ca.gov/dynamicapp/req/CSVDataServlet?Stations={}&SensorNums=15&dur_code=M&Start=2002-01-01&End=2017-12-31".format(i)
    urlData = requests.get(url).content
    df = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
    
    if df.empty:
        pass
    else:
        res_data[i] = df


In [ ]:
# Unpack the reservoir storage data and make a timeseries sum for the whole study area 

res_s = []
res_df = {}

for k,v in res_data.items():
    result = pd.to_numeric(res_data[k].VALUE, errors = "coerce")
    if len(result) != 192:
        pass
    else:
        res_s.append(pd.to_numeric(res_data[k].VALUE, errors = "coerce"))
        res_df[k] = pd.to_numeric(res_data[k].VALUE, errors = "coerce")

res_ts = np.nansum(np.column_stack(res_s), axis = 1)

# For plotting: Calculate monthly means
means = pd.DataFrame.from_dict(res_df).mean(axis = 0)
m = pd.DataFrame(means* 1.23348e-6, index = means.index) # Acre ft to cubic km
m.columns = ["storage"] 

In [ ]:
# for the other hydrology data, make a df starting 1 year earlier

years2 = range(2002, 2018)
months2 = range(1,13)
start2 = datetime.datetime(years2[0], 1, 1)
end2 = datetime.datetime(years2[-1]+1, 1, 1)
dt_idx2 = pd.date_range(start2,end2, freq='M')


df2 = pd.DataFrame([gldas_sm,tc_sm,gldas_swe,fldas_swe,res_ts*1.23348e-6]).T
df2.columns = ["gldas_sm", "tc_sm", "gldas_swe", "fldas_swe", "res_ts"]
df2.drop(df2.tail(1).index,inplace=True)
df2.index = dt_idx2

In [ ]:
## Now you have the Grace data in df and other dS data in df2 

# Merge them
fin_df = pd.merge(df, df2, how = "inner", left_index = True, right_index = True)

# Calculate mean SWE and SM 
fin_df['swe'] = np.mean([fin_df.gldas_swe, fin_df.fldas_swe], axis = 0)
fin_df['sm'] = np.mean([fin_df.tc_sm, fin_df.gldas_sm], axis = 0)

# Correct GRACE signal for sm, swe, reservoir storage 
fin_df['grace'] = fin_df.grace_median - fin_df.sm.diff()  - fin_df.swe.diff() - fin_df.res_ts.diff()

# Linear interpolation of missing values
fin_df['t'] = fin_df.grace.interpolate(method="linear")

# apply SavGol filter
fin_df['ts_savgol'] = savgol_filter(fin_df.t, window_length=23, polyorder=2)

In [ ]:
plt.plot(np.cumsum(fin_df.grace_median.diff()), label = "t")
# plt.plot(fin_df.mas)
# plt.plot(fin_df.cri)
plt.plot(fin_df.jpl)
# plt.plot(fin_df.gfz)
# plt.plot(np.cumsum(fin_df.grace.diff()))
plt.plot(fin_df.grace)
plt.plot(fin_df.ts_savgol)

plt.legend()

In [ ]:
fin_df.to_csv("../data/GRACE_extended.csv")